In [2]:
 
import pandas as pd

import numpy as np

orders = pd.read_csv('../staging_1/orders/orders.csv')

In [12]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432 entries, 0 to 1431
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       1432 non-null   int64         
 1   customer_id    1432 non-null   int64         
 2   order_status   1432 non-null   int64         
 3   order_date     1432 non-null   object        
 4   required_date  1432 non-null   datetime64[ns]
 5   shipped_date   1432 non-null   datetime64[ns]
 6   store_id       1432 non-null   int64         
 7   staff_id       1432 non-null   int64         
 8   extract_time   1432 non-null   object        
 9   source         1432 non-null   object        
dtypes: datetime64[ns](2), int64(5), object(3)
memory usage: 112.0+ KB


In [18]:
orders['required_date'] = pd.to_datetime(orders['required_date'])
orders['shipped_date'] = pd.to_datetime(orders['shipped_date'])
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [15]:
orders['late_delivery'] = orders ['shipped_date'] > orders ['required_date']

In [19]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432 entries, 0 to 1431
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       1432 non-null   int64         
 1   customer_id    1432 non-null   int64         
 2   order_status   1432 non-null   int64         
 3   order_date     1432 non-null   datetime64[ns]
 4   required_date  1432 non-null   datetime64[ns]
 5   shipped_date   1432 non-null   datetime64[ns]
 6   store_id       1432 non-null   int64         
 7   staff_id       1432 non-null   int64         
 8   extract_time   1432 non-null   object        
 9   source         1432 non-null   object        
 10  late_delivery  1432 non-null   bool          
dtypes: bool(1), datetime64[ns](3), int64(5), object(2)
memory usage: 113.4+ KB


In [24]:

orders['latency days'] = np.where(orders['shipped_date'] > orders['required_date'],
                                   (orders['shipped_date'] - orders['required_date']).dt.days,
                                   None)


In [26]:
orders.head(1)

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,extract_time,source,late_delivery,latency days
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-04-30 11:26:02.921696,postgres,False,None


In [27]:
orders['weekend'] = orders['shipped_date'].dt.dayofweek.isin([6, 0])


In [28]:
orders

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,extract_time,source,late_delivery,latency days,weekend
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-04-30 11:26:02.921696,postgres,False,None,True
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-04-30 11:26:02.921696,postgres,False,None,True
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-04-30 11:26:02.921696,postgres,False,None,True
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-04-30 11:26:02.921696,postgres,True,1,False
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-04-30 11:26:02.921696,postgres,False,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427,1474,816,4,2018-03-30,2018-04-01,2018-04-02,1,3,2024-04-30 11:26:02.921696,postgres,True,1,True
1428,1475,112,4,2018-03-30,2018-04-01,2018-03-31,2,6,2024-04-30 11:26:02.921696,postgres,False,None,False
1429,1476,35,4,2018-03-31,2018-04-01,2018-04-01,2,7,2024-04-30 11:26:02.921696,postgres,False,None,True
1430,1477,180,4,2018-03-31,2018-04-03,2018-04-01,2,6,2024-04-30 11:26:02.921696,postgres,False,None,True


In [29]:

status_mapping = {
    1: 'Pending',
    2: 'Processing',
    3: 'Rejected',
    4: 'Completed'
}

status_df = pd.DataFrame(list(status_mapping.items()), columns=['status_code', 'status_description'])


status_df



,status_code,status_description
0,1,Pending
1,2,Processing
2,3,Rejected
3,4,Completed


In [34]:
import os

output_path='../staging_2/order_status/order_status.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
status_df.to_csv(output_path, index=False)


In [ ]:
import pandas as pd
import os
output_path = '../staging_2/orders/orders.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
orders.to_csv(output_path, index=False)
